# Portfolio Optimization - Basic Concepts

In [1]:
# Import necessary libraries
import numpy as np

## Example 1.1: Computing Expected Portfolio Return and Variance

# The goal of this example is to calculate the expected return and variance
# for a portfolio given its weights, the expected returns of the assets,
# and the covariance matrix of the asset returns.

# Define the expected returns vector (mu)
mu = np.array([0.04, 0.06])
# Expected returns for two assets

# Define the covariance matrix (Sigma)
Sigma = np.array([
    [1, 0.7],
    [0.7, 2]
])
# Covariance matrix representing how asset returns move together

# Define the portfolio weights vector (w)
w = np.array([0.5, 0.5])
# Weights of the assets in the portfolio

# Compute expected portfolio return (mu_p)
mu_p = np.dot(w, mu)
# This represents the weighted average of the expected returns

# Compute expected portfolio variance (sigma_p_squared)
sigma_p_squared = np.dot(np.dot(w, Sigma), w.T)
# This calculation takes into account how the weights and the covariance
# between the assets impact the portfolio's overall risk

# Print the results
print(f"Expected Portfolio Return (mu_p): {mu_p}")
print(f"Expected Portfolio Variance (sigma_p_squared): {sigma_p_squared}")

# Conclusion:
# The expected portfolio return and variance are crucial metrics for assessing
# the performance and risk of a portfolio. In this example, the portfolio is expected
# to return 5% with a variance of 1.1, which helps in understanding the risk-return
# trade-off of the portfolio.


Expected Portfolio Return (mu_p): 0.05
Expected Portfolio Variance (sigma_p_squared): 1.1


# Step 1: Reading the Data

First, we need to import necessary libraries and load the data. We will use `pandas` for data manipulation and analysis. The `read_excel` function allows us to load the Excel file directly into a DataFrame. We specify the `index_col` parameter to use the first column as the index, which contains dates.

In [6]:
import pandas as pd

# Load the dataset
data_path = '../Assignment2/Data_2A_2023.xlsx'
data = pd.read_excel(data_path, index_col=0)

# Display the first few rows to understand its structure
data.head()

,AAPL,DIS,GE,GS,MSFT
Date,,,,,
2000-01-03,8.875351,2.136762,-3.069418,-6.237617,-0.160553
2000-01-04,-8.430973,5.857729,-4.000059,-6.298665,-3.378026
2000-01-05,1.463246,4.150207,-0.173555,-4.682763,1.054386
2000-01-06,-8.653747,-3.984829,1.336983,4.278944,-3.349816
2000-01-07,4.736852,-1.581027,3.872088,0.379921,1.306844


# Step 2.0: Filtering the Data

Our analysis focuses on data up until the end of 2015. Let's filter the DataFrame to include only this period. This step ensures that our computations for average returns and the covariance matrix are based on the correct time frame.


In [7]:
# Filter data to include only up to and including the year 2015
data_until_2015 = data[:'2015']

data_until_2015.tail()  # Display the last few rows to verify the filtering

,AAPL,DIS,GE,GS,MSFT
Date,,,,,
2015-12-24,-0.534017,0.284209,-0.387699,-0.262371,-0.268739
2015-12-28,-1.120066,1.313055,0.227049,-0.465854,0.502980
2015-12-29,1.797419,-0.158516,1.229768,1.051641,1.072391
2015-12-30,-1.305863,-0.691082,-0.735322,-0.828184,-0.424421
2015-12-31,-1.919517,-1.184853,0.322067,-0.977977,-1.473976


# Step 2.1: Computing Average Returns

The average return for each stock is a critical input for portfolio optimization. We calculate it by taking the mean of the returns for each stock over our selected period. This gives us an idea of the expected return on each stock.


In [8]:
# Compute the average returns for each stock
average_returns = data_until_2015.mean()

average_returns


AAPL    0.124552
DIS     0.056689
GE      0.019221
GS      0.050193
MSFT    0.027795
dtype: float64

# Step 2.2: Computing the Covariance Matrix

The covariance matrix is an essential component of portfolio optimization as it reflects the variability of returns between stocks. It helps in understanding how stocks move relative to each other, which is crucial for diversification and risk management.


In [9]:
# Compute the covariance matrix for the returns
covariance_matrix = data_until_2015.cov()

covariance_matrix


,AAPL,DIS,GE,GS,MSFT
AAPL,7.491433,1.820900,1.972199,2.460151,2.278628
DIS,1.820900,4.003684,2.076010,2.427742,1.796510
GE,1.972199,2.076010,3.924373,2.625809,1.783263
GS,2.460151,2.427742,2.625809,6.077517,2.323456
MSFT,2.278628,1.796510,1.783263,2.323456,4.027520


# Step 3: Allocating Initial Portfolio Weights

Now, we will allocate five cells for the portfolio weights. These weights represent the fraction of the total portfolio allocated to each asset. We will initialize these weights to 0.2, signifying that we start with an equal (20%) investment in each of the five stocks.


In [11]:
import numpy as np

# Allocate and initialize the portfolio weights to 20% for each of the five assets
initial_weights = np.array([0.2, 0.2, 0.2, 0.2, 0.2])

# Display the initial weights
initial_weights


array([0.2, 0.2, 0.2, 0.2, 0.2])

# Step 4: Mean-Variance Efficient Portfolio Optimization

To compute the mean-variance efficient portfolio, we aim to minimize the portfolio's variance subject to certain constraints. The optimization problem can be defined as follows:

\[ \min_w w^T\Sigma w \]

Subject to the constraints that the portfolio return equals some target return \( r_p \), and that the sum of the weights equals 1:

\[ w^T\mu = r_p \]
\[ w^T\iota = 1 \]

where:
- \( w \) is the vector of portfolio weights,
- \( \Sigma \) is the covariance matrix of asset returns,
- \( \mu \) is the vector of expected asset returns,
- \( r_p \) is the target portfolio return,
- \( \iota \) is a vector of ones (to represent the sum of weights).

We will use SciPy's optimization library to solve this problem. For the purpose of this example, let's assume the target return \( r_p \) is the average of the individual expected returns we calculated earlier.


In [13]:
from scipy.optimize import minimize

# Define the optimization problem
def portfolio_variance(weights, covariance_matrix):
    return weights.T @ covariance_matrix @ weights

# Define the constraints for the optimization
# The portfolio's weights must sum to 1
constraints = ({'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1})

# Let's assume the target return is the average of the average returns
target_return = average_returns.mean()

# The expected return of the portfolio equals the target return
# This will be used as an additional constraint if necessary
def portfolio_return(weights, expected_returns):
    return weights.T @ expected_returns - target_return

# Assuming the target return is the average return of the assets
additional_constraints = ({'type': 'eq', 'fun': lambda weights: portfolio_return(weights, average_returns)})

# Combine all constraints
all_constraints = [constraints, additional_constraints]

# Initial guess for the weights (equal weighting)
initial_guess = np.full(len(average_returns), 1/len(average_returns))

# Perform the optimization
optimized_result = minimize(
    fun=portfolio_variance,
    x0=initial_guess,
    args=(covariance_matrix,),
    method='SLSQP',
    constraints=all_constraints,
    bounds=[(0, 1) for _ in range(len(average_returns))]  # This bounds the weights to be between 0 and 1
)

optimized_weights = optimized_result.x
optimized_weights, portfolio_variance(optimized_weights, covariance_matrix)


(array([0.19607456, 0.33871315, 0.1956114 , 0.03633315, 0.23326774]),
 2.6195024836554603)

# Explanation

Optimized Portfolio Weights:
The optimization process has produced the following weights for each asset in the portfolio:

Weight for Asset 1 (e.g., AAPL): 19.61%

Weight for Asset 2 (e.g., DIS): 33.87%

Weight for Asset 3 (e.g., GE): 19.56%

Weight for Asset 4 (e.g., GS): 3.63%

Weight for Asset 5 (e.g., MSFT): 23.33%

These weights are the proportions of the total portfolio value that should be allocated to each asset in order to minimize the variance of the portfolio's return, given the constraints that the weights must sum to 1 (or 100%), and assuming the target return is the average return of these assets.

Optimized Portfolio Variance:
The portfolio variance resulting from these weights is 2.6195. This value represents the expected variability of the portfolio's returns. In the context of a mean-variance optimization framework, a lower variance is preferable because it implies lower risk or volatility.

# Step 5: Calculating the Target Portfolio Return (rp)

To set a target return for the portfolio optimization, we will calculate the average return of only those stocks that have a positive expected return. This is based on the assumption that we want to target a return rate that reflects the performance of the better-performing assets in our selection.


In [14]:
# Calculate the average of the stocks with positive average returns to use as the target return, rp
positive_average_returns = average_returns[average_returns > 0]
target_return_rp = positive_average_returns.mean()

# Display the target return rp
target_return_rp

0.0556897326187464

# Step 6: Determining the Mean-Variance Efficient Portfolio Weights

Using the target return \( r_p \) calculated from the positive average returns, we will now determine the mean-variance efficient portfolio weights. We will solve for the weights that minimize the portfolio variance while achieving the target return. This optimization process involves a solver that takes into account the constraints of having all portfolio weights sum up to 1 and achieving the specified target return.


In [15]:
# Perform the optimization with the new target return rp
# Adjust the constraint for the portfolio return to equal the new target return rp
constraints = [
    {'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1},  # The sum of weights must be 1
    {'type': 'eq', 'fun': lambda weights: portfolio_return(weights, average_returns) - target_return_rp}  # Target return constraint
]

# Perform the optimization using the same function and constraints, with the updated target return
optimized_result_rp = minimize(
    fun=portfolio_variance,
    x0=initial_guess,
    args=(covariance_matrix,),
    method='SLSQP',
    constraints=constraints,
    bounds=[(0, 1) for _ in range(len(average_returns))]  # Weights between 0 and 1
)

# Extract the optimized weights
optimized_weights_rp = optimized_result_rp.x

# Display the optimized weights
optimized_weights_rp


array([8.05901139e-01, 1.94098861e-01, 1.19348975e-15, 0.00000000e+00,
       1.94289029e-16])

# Computing Annualized Portfolio Returns

After finding the optimized portfolio weights, we will use these weights to compute the annualized returns of the portfolio from 2015 to 2022. We will then analyze the mean and variance of the annualized portfolio returns and compare them with the 'promised' mean and variance from the optimization solver to see if they match or are close to each other.


In [16]:
# Calculate the portfolio returns for the period 2015 to 2022
data_from_2016 = data['2016':]  # Data from 2016 (inclusive) to the end of the dataset
portfolio_returns = (data_from_2016 @ optimized_weights_rp).mean() * 252  # Annualizing the returns

# Calculate the portfolio variance for the period 2015 to 2022
portfolio_variance_annualized = (data_from_2016 @ optimized_weights_rp).var() * 252  # Annualizing the variance

# Display the annualized portfolio returns and variance
portfolio_returns, portfolio_variance_annualized


(23.59531620083109, 756.9485582373021)

# Explanation

Annualized Portfolio Returns:
Your portfolio's annualized return is 23.60%. This means that, on average, the portfolio would have yielded a return of 23.60% per year over the period of 2016 to 2022, based on the optimized weights from the solver.

Annualized Portfolio Variance:
The portfolio's annualized variance is 756.95. The variance is a measure of the spread of the portfolio's returns around the mean (annualized return). A higher variance indicates a higher level of risk or volatility. The square root of the variance, known as the standard deviation, is often used as a measure of risk.

# Step 7: Computing the Minimum Variance Portfolio (MVP)

The Minimum Variance Portfolio is the portfolio with the lowest risk, which in this context is measured as variance. To find the MVP, we perform an optimization similar to before, but this time without setting a target return constraint. The optimization will only ensure that the weights sum up to 1 (100% of the capital is invested).


In [17]:
# Perform the optimization to find the Minimum Variance Portfolio (MVP)
# We adjust the constraints to only ensure that the weights sum to 1
constraints = [{'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1}]

# Perform the optimization using the same function and constraints, without the target return constraint
mvp_result = minimize(
    fun=portfolio_variance,
    x0=initial_guess,
    args=(covariance_matrix,),
    method='SLSQP',
    constraints=constraints,
    bounds=[(0, 1) for _ in range(len(average_returns))]  # Weights between 0 and 1
)

# Extract the optimized weights for the MVP
mvp_weights = mvp_result.x

# Display the optimized weights for the MVP
mvp_weights


array([0.09044266, 0.28874646, 0.29391289, 0.02022814, 0.30666984])

# Analyzing Portfolio Performance: Training Period and Ex-Post

We will now analyze the performance of the Minimum Variance Portfolio during the training period (up to and including 2015) as well as ex-post (2016 to 2022). This will provide insights into how well the optimization worked out-of-sample and whether the minimized variance during the training period translated into lower actual variance ex-post.


In [18]:
# Calculate portfolio returns and variance during the training period using MVP weights
training_period_returns = (data_until_2015 @ mvp_weights).mean() * 252  # Annualize the returns
training_period_variance = (data_until_2015 @ mvp_weights).var() * 252  # Annualize the variance

# Calculate portfolio returns and variance ex-post using MVP weights
ex_post_returns = (data_from_2016 @ mvp_weights).mean() * 252  # Annualize the returns
ex_post_variance = (data_from_2016 @ mvp_weights).var() * 252  # Annualize the variance

# Display the annualized portfolio returns and variance for the training period and ex-post
training_period_returns, training_period_variance, ex_post_returns, ex_post_variance


(10.791103206745094, 637.8402580735755, 9.88157274863712, 598.5376246006648)